<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Kaggle%2017%20Flowers/Kaggle_17_Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import layers,optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset_path = '/content/drive/MyDrive/Flowers'
width = height = 224
batch_size = 32
data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
    rotation_range = 20,
    zoom_range = 0.1,
    validation_split = 0.2
)

In [5]:
train_data = data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
    subset= 'training'
)

Found 884 images belonging to 17 classes.


In [6]:
val_data = data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
    subset= 'validation'
)

Found 204 images belonging to 17 classes.


In [7]:
base_model = tf.keras.applications.VGG16(
    input_shape = (width,height,3),
    include_top = False,
    weights = 'imagenet',
)

58900480/58889256 [==============================] - 1s 0us/step


In [8]:
for layer in base_model.layers:
  layer.trainable = False

In [9]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
model = tf.keras.Sequential([
    base_model, 
    layers.Flatten(),
    layers.Dense(1024,activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(17,activation='softmax')
])

In [11]:
model.compile( optimizer = optimizers.Adam(learning_rate=0.001),
              loss = losses.categorical_crossentropy,
              metrics =  ['accuracy'])

In [12]:
model.fit(train_data,
          steps_per_epoch = train_data.samples/batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples/batch_size,
          epochs = 10,
          )

Epoch 1/10
27/27 [==============================] - 319s 10s/step - loss: 3.4546 - accuracy: 0.2817 - val_loss: 1.1841 - val_accuracy: 0.6912
Epoch 2/10
27/27 [==============================] - 22s 790ms/step - loss: 0.9653 - accuracy: 0.7149 - val_loss: 1.1038 - val_accuracy: 0.6961
Epoch 3/10
27/27 [==============================] - 22s 793ms/step - loss: 0.5066 - accuracy: 0.8382 - val_loss: 0.6646 - val_accuracy: 0.8137
Epoch 4/10
27/27 [==============================] - 22s 791ms/step - loss: 0.2761 - accuracy: 0.9253 - val_loss: 0.5564 - val_accuracy: 0.8235
Epoch 5/10
27/27 [==============================] - 22s 791ms/step - loss: 0.1830 - accuracy: 0.9593 - val_loss: 0.5599 - val_accuracy: 0.8431
Epoch 6/10
27/27 [==============================] - 22s 786ms/step - loss: 0.1887 - accuracy: 0.9559 - val_loss: 0.5046 - val_accuracy: 0.8676
Epoch 7/10
27/27 [==============================] - 22s 782ms/step - loss: 0.1301 - accuracy: 0.9740 - val_loss: 0.4998 - val_accuracy: 0.8431


In [13]:
test_data = data_generator.flow_from_directory(
    os.path.join(dataset_path,'Test'),
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
    )

Found 272 images belonging to 17 classes.


In [14]:
model.evaluate(test_data)

9/9 [==============================] - 66s 8s/step - loss: 1.2259 - accuracy: 0.6471


[1.225874423980713, 0.6470588445663452]